In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
Dataset contains more than 2000 job listing for data analyst positions, with different Job Title names such as:

Target:
Job Title(Diff data analyst)
Salary
Loction(City/State)
Rating


In [ ]:
# Data Manuplation
import pandas as pd
import numpy as np

# Data Visualization
import matplotlib.pyplot as plt 
import seaborn as sns
import plotly.express as px
import plotly.graph_objs as go

import re

%matplotlib inline

#Ignore warnings
import warnings
warnings.filterwarnings('ignore')


In [ ]:
df = pd.read_csv('../input/data-analyst-jobs/DataAnalyst.csv')
df.head()

In [ ]:
df.shape

In [ ]:
df.info()

In [ ]:
df.values

In [ ]:
# Dropping Columns from DataFrame
df.drop(['Unnamed: 0','Job Description',"Company Name",'Headquarters', 'Size', 'Founded','Type of ownership','Revenue','Competitors','Easy Apply'], 1,  inplace=True)

In [ ]:
# Renaming Column names
df.rename(columns={ "Job Title":"Job_Title", 
                  "Salary Estimate": "Salary_Estimate"}, inplace=True)


In [ ]:
df.head()

Data Preparation

In [ ]:
# Splitting  Job_Title into understandable
split_Job_Title= df.Job_Title.str.split(' -|—|, ', expand= True)

split_Job_Title

In [ ]:
# Cleaned Job_Title column
split_Job_Title = split_Job_Title.replace({'Data Analyst 2': 'Data Analyst II',
                                           'Data Analyst 3': 'Data AnalystIII',
                                           'RQS': 'Data Security Analyst',
                                           'JPSC-7176':'Senior Data Analyst',
                                           'Data Analyst *Temp-Hire*':'Data Analyst',
                                           'Data Analyst-IT II':'Data Analyst II',
                                           'IT Data Analyst':'Data Analyst',
                                           'CCB Risk':'Fraud Risk Data Analyst','Analyst':'Analyst Data Intelligence',
                                           'CDHP': 'Data Analyst Team Lead','ACA':'Data Analyst','VP':'Research Project Analyst/Data Management',
                                           'JPSC-8461':'Data Analyst','.net': 'Business Analyst\Data Analyst','.Net Developer':'Data Design Analyst',
                                           "Technical Business Analyst (SQL": 'Technical Business Analyst (SQL Data analytics ETL)',
                                           'Data Systems Analyst (7199U':'Data Systems Analyst',
                                            'Sr. Data Analyst': 'Senior Data Analyst', 
                                            'Sr Data Analyst': 'Senior Data Analyst',
                                            'Data Analyst Junior': 'Junior Data Analyst',
                                          '#104252 Division Data and Financial Analyst': 'Data and Finanical Analyst',
                                            '#105009 Research Data Analyst':'Research Data Analyst',
                                           '(Data Analyst) BA/BSA':'Data Analyst','(REMOTE) Sr. Risk Analyst (IT}':'Senior Risk Analyst',
                                           '(Risk Data Analyst)':"Risk Data Analyst",'(Data Risk Analyst: Data Modeler)':'Risk Data Analyst'})

In [ ]:
# Salary Column Cleaning
df['Salary_Estimate'] = df['Salary_Estimate'].apply(lambda x: '$0K-$0K' if x == '-1' else x)
df['Salary_Estimate'] = df['Salary_Estimate'].apply(lambda x: x.split('(')[0])

salary= df['Salary_Estimate'].str.split('-', expand=True)
df['MinSalary($/k)'] = pd.to_numeric(salary[0].str.extract('(\d+)', expand=False))
df['MaxSalary($/k)'] = pd.to_numeric(salary[1].str.extract('(\d+)', expand=False))

df.head()

In [ ]:
#df.drop(['Salary_Estimate',"MinSalary",'MaxSalary','MinSalary($)'],axis=1,inplace=True)
df.drop(['Salary_Estimate'],axis=1,inplace=True)

In [ ]:
# Industry Column Cleaning
df['Industry'] = df['Industry'].apply(lambda x: 'NAN' if x == '-1' else x)
df['Industry'].value_counts().head()

In [ ]:
# Sector column Cleaning
df['Sector'] = df['Sector'].apply(lambda x: 'NAN' if x == '0' else x)
df['Sector'].value_counts().head()

In [ ]:
# split into state & City
split_Location= df.Location.str.split(',',expand=True)
split_Location

In [ ]:
# Assign Coulmn names
split_Location.columns=['City','State','None']

split_Location

In [ ]:
# Adding columns to our output DataFrame
df= pd.concat([df, split_Location,split_Job_Title], axis = 1)

In [ ]:
df.head()

In [ ]:
df.drop(['Job_Title','Location','None',1,2,3,4,5],1, inplace= True)

In [ ]:
# Rename Job_title after cleaning
df.rename(columns={0:'Job_Title'}, inplace=True)

In [ ]:

df.head()

> > **Top 20 Job Titles**

In [ ]:
popular_titles= df['Job_Title'].value_counts().nlargest(n=20)
#popular_titles= popular_titles[popular_titles==True]
#popular_titles.plot()
popular_titles

In [ ]:
plt.figure(figsize=(10,5))

ax=sns.barplot(x=df['Job_Title'].value_counts()[0:20],y=df['Job_Title'].value_counts()[0:20].index)
plt.title('Top 20 Job_Titles by Job Posts')
plt.ylabel('')
plt.xlabel('')
plt.show()

In [ ]:
plt.figure(figsize=(10,5))
ax=sns.barplot(x=df['MinSalary($/k)'].value_counts()[0:20],y=df['MinSalary($/k)'].value_counts()[0:20].index)
plt.title('Salary_Estimate by Glassdoor')
plt.ylabel('')
plt.xlabel('')
plt.show()

Bar plot gives us an estimate of central tendency for a numeric variable (height of each rectangle) and an indication of the 

uncertainty around that estimate (error bars in black).

In [ ]:
 #Transform into categorical
df['Job_Title'] = pd.Categorical(df['Job_Title'])
df.dtypes
# Plot bar plot (titles, age and sex)
plt.figure(figsize=(15,5))
ax=sns.barplot(x=df['Job_Title'].value_counts()[0:20],y=df['City'].value_counts()[0:20].index)


In [ ]:
Sector = df.groupby(['Sector']).mean()

fig = go.Figure()

fig.add_trace(go.Bar(x=Sector.index,y=Sector['MinSalary($/k)'],name='Mininum Mean Salary',text=Sector['MinSalary($/k)'],textposition='auto' ))
fig.add_trace(go.Bar(x=Sector.index,y=Sector['MaxSalary($/k)'],name='Maximum Mean Salary',text=Sector['MinSalary($/k)'],textposition='auto'))
fig.update_layout(title='Mean Salary',barmode='stack')
fig.show()

In [ ]:
location = df[df['Job_Title']=='Data Analyst']['State'].value_counts()
fig = px.pie(location,names=location.index,values=location,height=600)
fig.update_traces(textposition='inside',textinfo='label+percent',hole=.3)

fig.show()

In [ ]:
role = df['Job_Title'].value_counts().nlargest(n=15)
fig = px.pie(role, values = role.values, names = role.index,  title="Top 15 Job_Titles", color=role.values,
       color_discrete_sequence=px.colors.qualitative.Prism)
fig.update_traces(opacity=0.7,marker_line_color='rgb(8,48,107)',marker_line_width=1.5)
fig.update_layout(title_x=0.5)
fig.show()

In [ ]:
z=df.groupby(['Job_Title']).mean()
z.iloc[20:30]

In [ ]:
df.plot.scatter(x='MinSalary($/k)', y='State',figsize=(10,5))

In [ ]:
f,axes= plt.subplots(1,2, sharey=False, figsize=(10,5), dpi=100, facecolor= 'white')

table1= pd.pivot_table(df, index='State', columns='MaxSalary($/k)', values='Rating').plot(kind='bar', ax=axes[0])
table1= pd.pivot_table(df, index='State', columns='MaxSalary($/k)', values='MinSalary($/k)').plot(kind='bar', ax=axes[1])

axes[0].set_ylabel('Rating_score')
axes[1].set_ylabel('Basic_Salary')
axes[0].set_ylim((0,9.9))
axes[0].set_ylim((0,1.4));


In [ ]:
df.sort_values(['Rating','MaxSalary($/k)'], ascending=(True, False), inplace= None)
df.head(10)

In [ ]:
### Univarient Analysis

In [ ]:
df.groupby(['Job_Title']).count()

In [ ]:
freq_table= df.groupby(['Job_Title']).size().reset_index(name='Count')
freq_table.head()

In [ ]:
freq_table['Counts']=freq_table['Count']/sum(freq_table['Count'])*100
freq_table

In [ ]:
freq_table= df.groupby(['Rating']).size().reset_index(name='Count')
freq_table.head(10)

In [ ]:
plt.figure(figsize=(10,5))
x = df['Rating']
plt.hist(x, bins=20, color='green')
plt.title('Rating of companies')
plt.xlabel('Rating')
plt.ylabel('Count')
plt.show()


In [ ]:
plt.figure(figsize=(10,5))
df.boxplot()

In [ ]:
df['MinSalary($/k)'].median()

In [ ]:
np.percentile(df['MaxSalary($/k)'],75)